In [7]:
import numpy as np
import cv2
import mahotas
import os
import sys
import pickle
import joblib  # Corrected import of joblib

# Configure file paths and parameters
TEST_FOLDER = "test"  # Path to your test folder containing images
BOVW = "bovw_codebook_600.pickle"
MODEL = 'rfclassifier_600.sav'
IMG_SIZE = 320

# Hu Moments
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

# Haralick Texture
def fd_haralick(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick

# Color Histogram
def fd_histogram(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    bins = 8
    hist = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

# SIFT Bag of Visual Words
def feature_extract(im):
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()  # Use cv2.SIFT_create() if you're using OpenCV 4.4.0+
    keypoints = sift.detect(gray)
    feature = bowDiction.compute(gray, keypoints)
    return feature.squeeze()

# Load the trained model
try:
    loaded_model = joblib.load(MODEL)
    print(f"Model loaded successfully from {MODEL}")
except Exception as e:
    print(f"Error loading model: {e}")

# Load the BOVW codebook
with open(BOVW, "rb") as pickle_in:
    dictionary = pickle.load(pickle_in)

# Initialize SIFT BOW image descriptor extractor
sift2 = cv2.SIFT_create()  # OpenCV 4.4.0+ use SIFT_create directly
bowDiction = cv2.BOWImgDescriptorExtractor(sift2, cv2.BFMatcher(cv2.NORM_L2))
bowDiction.setVocabulary(dictionary)

# Class-label dictionary
label = {0: "10", 1: "20", 2: "50", 3: "100", 4: "200", 5: "500", 6: "2000"}

# Loop through all images in the test folder
for image_filename in os.listdir(TEST_FOLDER):
    image_path = os.path.join(TEST_FOLDER, image_filename)

    # Skip non-image files
    if not image_filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue
    
    # Read the image
    image = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if image is None:
        print(f"Error: The image at {image_path} could not be loaded. Skipping this file.")
        continue

    # Resize the image
    (height, width, channel) = image.shape
    resize_ratio = 1.0 * (IMG_SIZE / max(width, height))
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    input_image = cv2.resize(image, target_size)

    # Print the current image being processed
    print(f"Processing image: {image_filename}")

    # Extract the features
    Hu = fd_hu_moments(input_image)
    Harl = fd_haralick(input_image)
    Hist = fd_histogram(input_image)
    Bovw = feature_extract(input_image)

    # Generate a feature vector by combining all features
    mfeature = np.hstack([Hu, Harl, Hist, Bovw])

    # Predict the output using the trained model
    try:
        output = loaded_model.predict(mfeature.reshape((1, -1)))
        print(f"Predicted class for {image_filename}: " + label[output[0]])
    except Exception as e:
        print(f"Error during prediction for {image_filename}: {e}")


Model loaded successfully from rfclassifier_600.sav
Processing image: 10.jpg


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.1s finished


Predicted class for 10.jpg: 10
Processing image: 100.jpg


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.1s finished


Predicted class for 100.jpg: 100
Processing image: 200.jpg


...

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.0s finished


Predicted class for 2000.jpg: 2000
Processing image: 500.jpg


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.0s finished


Predicted class for 500.jpg: 500
Processing image: currency_test.jpg
Predicted class for currency_test.jpg: 100


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.0s finished
